In [2]:
'''CODE HERE'''
import pandas as pd
#produces a prediction model in the form of an ensemble of weak prediction models, typically decision tree
# import xgboost as xgb
#the outcome (dependent variable) has only a limited number of possible values. 
#Logistic Regression is used when response variable is categorical in nature.
# from sklearn.linear_model import LogisticRegression
# #A random forest is a meta estimator that fits a number of decision tree classifiers 
# #on various sub-samples of the dataset and use averaging to improve the predictive 
# #accuracy and control over-fitting.
# from sklearn.ensemble import RandomForestClassifier
# #a discriminative classifier formally defined by a separating hyperplane.
# from sklearn.svm import SVC
# #displayd data
# from IPython.display import display
# %matplotlib inline




In [9]:

data = pd.read_csv('Data/finalData.csv')
# Preview data.
data.sort_values(by=['Date'])
print(data)

     Div      Date        HomeTeam        AwayTeam  FTHG  FTAG FTR  HTHG  \
0     E0  16/08/08         Arsenal       West Brom   1.0   0.0   H   1.0   
1     E0  16/08/08          Bolton           Stoke   3.0   1.0   H   3.0   
2     E0  16/08/08         Everton       Blackburn   2.0   3.0   A   1.0   
3     E0  16/08/08            Hull          Fulham   2.0   1.0   H   1.0   
4     E0  16/08/08   Middlesbrough       Tottenham   2.0   1.0   H   0.0   
5     E0  16/08/08      Sunderland       Liverpool   0.0   1.0   A   0.0   
6     E0  16/08/08        West Ham           Wigan   2.0   1.0   H   2.0   
7     E0  17/08/08     Aston Villa        Man City   4.0   2.0   H   0.0   
8     E0  17/08/08         Chelsea      Portsmouth   4.0   0.0   H   3.0   
9     E0  17/08/08      Man United       Newcastle   1.0   1.0   D   1.0   
10    E0  23/08/08       Blackburn            Hull   1.0   1.0   D   1.0   
11    E0  23/08/08          Fulham         Arsenal   1.0   0.0   H   1.0   
12    E0  23